In [2]:
import pandas as pd
import numpy as np
import random
import datetime
import boto3
import botocore
import psycopg2
import configparser

### Nombre de la instancia

In [3]:
rdsIdentifier = 'dpir'

### Cargar archivo de config.

In [4]:
config = configparser.ConfigParser()
config.read('proyecto.cfg')

['proyecto.cfg']

### Crear instancia

In [5]:
aws_data = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                     aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                     region_name='us-east-1')

### Instancias disponibles

In [6]:
rdsInstanceIds = []

response = aws_data.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['dpir']


### Creacion de servicio RDS

In [6]:
try:
    response = aws_data.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=["sg-02abf94b67ee35a24"],
            PubliclyAccessible=True
        )
    print (response)
except aws_data.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

{'DBInstance': {'DBInstanceIdentifier': 'dpir', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'banco_admin', 'DBName': 'banco', 'AllocatedStorage': 10, 'PreferredBackupWindow': '05:59-06:29', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-02abf94b67ee35a24', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres14', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-03c402afa5aa4731d', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-0e4d6e3233e29322c', 'SubnetAvailabilityZone': {'Name': 'us-east-1a'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0ca92c970d882f987', 'SubnetAvailabilityZone': {'Name': 'us-east-1b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0a07cf47

### Obteniendo URL del host

In [7]:
try:
     instances = aws_data.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dpir.cznwk1uf05mh.us-east-1.rds.amazonaws.com


### Cargar informacion a base de datos

In [24]:
import DDLPR_V16

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(DDLPR_V16.create_pr_dw)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de Datos Creada Exitosamente


### Funcion para cargar datos

In [16]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

In [37]:
productos = pd.read_excel('productos.xlsx')


In [44]:
productos_dict = productos.to_dict(orient='records')


In [46]:
data_DIM_productos = productos_dict
insertDataToSQL(data_DIM_productos, 'DIM_Productos')


Se han insertado 158 nuevos registros.


In [55]:
fechas = pd.read_excel('dim_date.xlsx')
fechas_dict = fechas.to_dict(orient='records')
data_DIM_Date = fechas_dict
insertDataToSQL(data_DIM_Date, 'DIM_Date')


Se han insertado 1000 nuevos registros.


In [57]:
proveedor = pd.read_excel('dim_proveedor.xlsx')
proveedor_dict = proveedor.to_dict(orient='records')
data_dim_proveedor = proveedor_dict
insertDataToSQL(data_dim_proveedor, 'DIM_Proveedor')


Se han insertado 130 nuevos registros.


In [58]:
fuentes = pd.read_excel('dim_fuentes.xlsx')
fuentes_dict = fuentes.to_dict(orient='records')
data_dim_fuentes = fuentes_dict
insertDataToSQL(data_dim_fuentes, 'DIM_Fuente')

Se han insertado 6 nuevos registros.


In [60]:
rubro = pd.read_excel('dim_rubro.xlsx')
rubro_dict = rubro.to_dict(orient='records')
data_dim_rubro = rubro_dict
insertDataToSQL(data_dim_fuentes, 'DIM_Rubro')

Se han insertado 6 nuevos registros.


In [61]:
medidas = pd.read_excel('dim_medidas.xlsx')
medidas_dict = medidas.to_dict(orient='records')
data_dim_medidas = medidas_dict
insertDataToSQL(data_dim_medidas, 'DIM_Medida')

Se han insertado 5 nuevos registros.


In [62]:
entidad = pd.read_excel('dim_entidad.xlsx')
entidad_dict = entidad.to_dict(orient='records')
data_dim_entidad = entidad_dict
insertDataToSQL(data_dim_entidad, 'DIM_Entidad')

Se han insertado 3 nuevos registros.


In [63]:
adquisicion = pd.read_excel('dim_adquisicion.xlsx')
adquisicion_dict = adquisicion.to_dict(orient='records')
data_dim_adquisicion = adquisicion_dict
insertDataToSQL(data_dim_adquisicion, 'DIM_Adquisicion')

Se han insertado 3 nuevos registros.


In [64]:
divisas = pd.read_excel('dim_divisas.xlsx')
divisas_dict = divisas.to_dict(orient='records')
data_dim_divisas = divisas_dict
insertDataToSQL(data_dim_divisas, 'DIM_Divisas')

Se han insertado 8 nuevos registros.


In [ ]:
divisas = pd.read_excel('dim_divisas.xlsx')
divisas_dict = divisas.to_dict(orient='records')
data_dim_divisas = divisas_dict
insertDataToSQL(data_dim_divisas, 'DIM_Divisas')

In [65]:
precios = pd.read_excel('fact_precios.xlsx')
precios_dict = precios.to_dict(orient='records')
data_dim_precios = precios_dict
insertDataToSQL(data_dim_precios, 'FACT_Precio')

Se han insertado 1000 nuevos registros.


In [66]:
eventos = pd.read_excel('dim_evento.xlsx')
eventos_dict = eventos.to_dict(orient='records')
data_dim_eventos = eventos_dict
insertDataToSQL(data_dim_eventos, 'DIM_Evento')

Se han insertado 2 nuevos registros.


In [67]:
catalogo = pd.read_excel('fact_catalogo.xlsx')
catalogo_dict = catalogo.to_dict(orient='records')
data_dim_catalogo = catalogo_dict
insertDataToSQL(data_dim_catalogo, 'FACT_Catalogo')

Se han insertado 158 nuevos registros.


In [85]:
RDS_HOST = 'dpir.cznwk1uf05mh.us-east-1.rds.amazonaws.com'
print(RDS_HOST)

dpir.cznwk1uf05mh.us-east-1.rds.amazonaws.com


In [84]:
config = configparser.ConfigParser()
config.read('proyecto.cfg')

['proyecto.cfg']

In [86]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""

In [98]:
sql_query = 'select * from "DIM_Productos" dp '
df_Productos = pd.read_sql(sql_query, postgres_driver)
df_Productos.head(5)

,Producto_FK,Cod_INE,Nombre,Especificaciones,timestamp
0,1,295-491-000,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,2023-04-06 01:28:12
1,2,295-492-000,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,2023-04-06 01:28:12
2,3,295-493-000,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,2023-04-06 01:28:12
3,4,295-494-000,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,2023-04-06 01:28:12
4,5,295-495-000,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,SUTURA ABSORBIBLE NATURAL CRÓMICA (CATGUT CRÓM...,2023-04-06 01:28:12


In [99]:
sql_query = 'select * from "DIM_Adquisicion" da  '
df_Adquisicion = pd.read_sql(sql_query, postgres_driver)
df_Adquisicion

,Adquisicion_FK,Tipo_Adquisicion,timestamp
0,1,Contrato Abierto,2023-04-05 17:21:41
1,2,Subasta Electrónica Inversa,2023-04-05 17:21:41
2,3,Bienes y Suministros Importados,2023-04-05 17:21:41


In [101]:
sql_query = 'select * from "DIM_Date" dd'
df_Date = pd.read_sql(sql_query, postgres_driver)
df_Date.head()

,Date_key,Fecha,Mes,Año,Mes_Año,Fecha +40d
0,42005,2015-01-01,enero,2015,1.2015,2015-02-10
1,42006,2015-01-02,enero,2015,1.2015,2015-02-11
2,42007,2015-01-03,enero,2015,1.2015,2015-02-12
3,42008,2015-01-04,enero,2015,1.2015,2015-02-13
4,42009,2015-01-05,enero,2015,1.2015,2015-02-14


In [102]:
sql_query = 'select * from "DIM_Divisas" dd'
df_Divisas = pd.read_sql(sql_query, postgres_driver)
df_Divisas.head()

,Divisa_FK,Divisa,Abreviatura,Símbolo,timestamp
0,8,Quetzal Guatemalteco,GTQ,Q,2023-04-05 17:44:03
1,9,Dólar Estadounidense,USD,$,2023-04-05 17:44:03
2,10,Euro,EUR,€,2023-04-05 17:44:03
3,11,Peso Mexicano,MXN,$,2023-04-05 17:44:03
4,12,Peso Filipino,PHP,₱,2023-04-05 17:44:03


In [103]:
sql_query = 'select * from "DIM_Evento" de'
df_Evento = pd.read_sql(sql_query, postgres_driver)
df_Evento

,Evento_FK,NOG,Nombre,Enlace,Entidad,Adquisición_FK,Date_Key
0,1,18795668,CONTRATO ABIERTO NÚMERO 06-2022 ADQUISICIÓN DE...,Perla,2,1,44968
1,2,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,1,44966


In [104]:
sql_query = 'select * from "DIM_Fuente" df'
df_Fuente = pd.read_sql(sql_query, postgres_driver)
df_Fuente

,Fuente_FK,FuenteDatos,timestamp
0,1,Cotizaciones,2023-04-06 01:26:56
1,2,Ofertas CA,2023-04-06 01:26:56
2,3,Guatecompras,2023-04-06 01:26:56
3,4,PR anteriores,2023-04-06 01:26:56
4,5,Internacionales,2023-04-06 01:26:56
5,6,FEL,2023-04-06 01:26:56


In [105]:
sql_query = 'select * from "DIM_Medida" dm'
df_Medida = pd.read_sql(sql_query, postgres_driver)
df_Medida

,Medida_FK,UnidadMedida,Tipo,Dimensional,timestamp
0,1,UNIDAD,Unidades,u,2023-04-06 01:26:20
1,2,Kilogramo,Peso,kg,2023-04-06 01:26:20
2,3,Litro,Volumen,L,2023-04-06 01:26:20
3,4,gramo,Peso,g,2023-04-06 01:26:20
4,5,Mililitro,Volumen,ml,2023-04-06 01:26:20


In [107]:
sql_query = 'select * from "DIM_Proveedor" dp'
df_Proveedor = pd.read_sql(sql_query, postgres_driver)
df_Proveedor.head()

,Proveedor_FK,NIT,Nombre,Direccion,Teléfono,e-mail,Comentarios,Sector,timestamp
0,379,56711151,LaCorium Health USA Inc.,60 Warbler Junction,8754-3430,mjirieck0@ihg.com,None,Comercializadora,2023-04-07 01:43:52
1,380,30976393,"Navarro Discount Pharmacies,LLC",26 Wayridge Court,9579-4380,pvials1@istockphoto.com,None,Distribuidora,2023-04-07 01:43:52
2,381,56441208,Clinical Solutions Wholesale,83 Stephen Trail,3293-3088,cdinneen2@mapquest.com,None,Comercializadora,2023-04-07 01:43:52
3,382,27449569,H E B,205 Merrick Point,1934-0405,mkirkby3@army.mil,None,Distribuidora,2023-04-07 01:43:52
4,383,32574995,Direct RX,3939 Clarendon Avenue,3114-6303,dreadshaw4@berkeley.edu,None,Consumidor final,2023-04-07 01:43:52


In [192]:
sql_query = 'select * from "DIM_Entidad" de'
df_Entidad = pd.read_sql(sql_query, postgres_driver)
df_Entidad

,Entidad_FK,Nombre,Tipo,Sub_Tipo,Unidad_Compradora,timestamp
0,1,MINISTERIO DE SALUD PÚBLICA,Administración Central,None,None,2023-04-06 01:09:14
1,2,MINISTERIO DE FINANZAS PÚBLICAS,Administración Central,None,None,2023-04-06 01:09:14
2,3,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -I,Administración Central,None,None,2023-04-06 01:09:14


In [162]:
sql_query = 'select * from "DIM_Rubro" dr'
df_Rubro = pd.read_sql(sql_query, postgres_driver)
df_Rubro.head()

,Fuente_FK,FuenteDatos,timestamp
0,1,Cotizaciones,2023-04-06 01:26:56
1,2,Ofertas CA,2023-04-06 01:26:56
2,3,Guatecompras,2023-04-06 01:26:56
3,4,PR anteriores,2023-04-06 01:26:56
4,5,Internacionales,2023-04-06 01:26:56


In [111]:
sql_query = 'select * from "FACT_Precio" fp'
df_FACT_Precio = pd.read_sql(sql_query, postgres_driver)
df_FACT_Precio.head()

,Precio_ID,Producto_FK,Proveedor_FK,Fuente_FK,Date_key,Divisa_FK,Precio,Marca,Pais,NOG,Responsable,Observaciones,timestamp
0,1,58,413,3,42627,8,9.15,None,None,None,None,None,2023-04-07 02:00:33
1,2,5,383,3,42632,8,13.75,None,None,None,None,None,2023-04-07 02:00:33
2,3,5,391,3,42632,8,8.13,None,None,None,None,None,2023-04-07 02:00:33
3,4,5,384,3,42632,8,7.00,None,None,None,None,None,2023-04-07 02:00:33
4,5,8,384,3,42634,8,10.00,None,None,None,None,None,2023-04-07 02:00:33


### Hacer joins

In [116]:
df_FACT_Precio_Proveedor = df_FACT_Precio.merge(df_Proveedor, on='Proveedor_FK',how='inner')


In [126]:
df_Catalogo_Evento = df_Catalogo.merge(df_Evento, on ='Evento_FK', how='inner')
df_Catalogo_Evento = df_Catalogo_Evento.drop('Evento_FK', axis = 1)
df_Catalogo_Evento.head(1)


,Catalogo_ID,Revision,Renglon,Producto_FK,Medida_FK,Cantidad,Rubro_FK,timestamp,PR,Date_key,Observaciones,NOG,Nombre,Enlace,Entidad,Adquisición_FK,Date_Key
0,1,1,1,154,3,7700,13,2023-04-06 12:47:32,None,None,None,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,1,44966


In [131]:
df_Catalogo_Evento_Producto = df_Catalogo_Evento.merge(df_Productos, on='Producto_FK',how='inner')
df_Catalogo_Evento_Producto = df_Catalogo_Evento_Producto.drop('Producto_FK',axis=1)
df_Catalogo_Evento_Producto.head(2)

,Catalogo_ID,Revision,Renglon,Medida_FK,Cantidad,Rubro_FK,timestamp_x,PR,Date_key,Observaciones,NOG,Nombre_x,Enlace,Entidad,Adquisición_FK,Date_Key,Cod_INE,Nombre_y,Especificaciones,timestamp_y
0,1,1,1,3,7700,13,2023-04-06 12:47:32,None,None,None,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,1,44966,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,2023-04-06 01:28:12
1,2,1,2,2,3850,13,2023-04-06 12:47:32,None,None,None,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,1,44966,264-002-000,PLAGUICIDA BIOLÓGICO BACILLUS SPHAERICUS,PLAGUICIDA BIOLÓGICO PARA CONTROL DE LARVAS AN...,2023-04-06 01:28:12


In [136]:
df_Catalogo_Evento_Producto_Medida = df_Catalogo_Evento_Producto.merge(df_Medida, on='Medida_FK', how='inner')
df_Catalogo_Evento_Producto_Medida = df_Catalogo_Evento_Producto_Medida.drop('Medida_FK', axis=1)
df_Catalogo_Evento_Producto_Medida.head(2)

,Catalogo_ID,Revision,Renglon,Cantidad,Rubro_FK,timestamp_x,PR,Date_key,Observaciones,NOG,...,Adquisición_FK,Date_Key,Cod_INE,Nombre_y,Especificaciones,timestamp_y,UnidadMedida,Tipo,Dimensional,timestamp
0,1,1,1,7700,13,2023-04-06 12:47:32,None,None,None,18558690,...,1,44966,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,2023-04-06 01:28:12,Litro,Volumen,L,2023-04-06 01:26:20
1,2,1,2,3850,13,2023-04-06 12:47:32,None,None,None,18558690,...,1,44966,264-002-000,PLAGUICIDA BIOLÓGICO BACILLUS SPHAERICUS,PLAGUICIDA BIOLÓGICO PARA CONTROL DE LARVAS AN...,2023-04-06 01:28:12,Kilogramo,Peso,kg,2023-04-06 01:26:20


### Renombrando columna Adquisicion

In [150]:
df_Catalogo_Evento_Producto_Medida = df_Catalogo_Evento_Producto_Medida.rename(columns={'Adquisición_FK':'Adquisicion_FK'})
df_Catalogo_Evento_Producto_Medida.head(2)

,Catalogo_ID,Revision,Renglon,Cantidad,Rubro_FK,timestamp_x,PR,Date_key,Observaciones,NOG,...,Adquisicion_FK,Date_Key,Cod_INE,Nombre_y,Especificaciones,timestamp_y,UnidadMedida,Tipo,Dimensional,timestamp
0,1,1,1,7700,13,2023-04-06 12:47:32,None,None,None,18558690,...,1,44966,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,2023-04-06 01:28:12,Litro,Volumen,L,2023-04-06 01:26:20
1,2,1,2,3850,13,2023-04-06 12:47:32,None,None,None,18558690,...,1,44966,264-002-000,PLAGUICIDA BIOLÓGICO BACILLUS SPHAERICUS,PLAGUICIDA BIOLÓGICO PARA CONTROL DE LARVAS AN...,2023-04-06 01:28:12,Kilogramo,Peso,kg,2023-04-06 01:26:20


In [161]:
df_Catalogo_Evento_Producto_Medida_Ad = df_Catalogo_Evento_Producto_Medida.merge(df_Adquisicion, on='Adquisicion_FK', how='inner')
df_Catalogo_Evento_Producto_Medida_Ad.head(2)

C:\Users\Marlon Daniel\AppData\Local\Temp\ipykernel_5460\2794142737.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'timestamp_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_Catalogo_Evento_Producto_Medida_Ad = df_Catalogo_Evento_Producto_Medida.merge(df_Adquisicion, on='Adquisicion_FK', how='inner')


,Catalogo_ID,Revision,Renglon,Cantidad,Rubro_FK,timestamp_x,PR,Date_key,Observaciones,NOG,...,Cod_INE,Nombre_y,Especificaciones,timestamp_y,UnidadMedida,Tipo,Dimensional,timestamp_x,Tipo_Adquisicion,timestamp_y
0,1,1,1,7700,13,2023-04-06 12:47:32,None,None,None,18558690,...,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,2023-04-06 01:28:12,Litro,Volumen,L,2023-04-06 01:26:20,Contrato Abierto,2023-04-05 17:21:41
1,2,1,2,3850,13,2023-04-06 12:47:32,None,None,None,18558690,...,264-002-000,PLAGUICIDA BIOLÓGICO BACILLUS SPHAERICUS,PLAGUICIDA BIOLÓGICO PARA CONTROL DE LARVAS AN...,2023-04-06 01:28:12,Kilogramo,Peso,kg,2023-04-06 01:26:20,Contrato Abierto,2023-04-05 17:21:41


## Limpiando columnas

In [172]:
df_Catalogo_Evento_Producto_Medida_Ad = df_Catalogo_Evento_Producto_Medida_Ad.drop(['timestamp_y','timestamp_y','Observaciones'],axis=1)

In [175]:
df_Catalogo_Evento_Producto_Medida_Ad.head(3)

,Catalogo_ID,Revision,Renglon,Cantidad,NOG,Nombre_x,Enlace,Entidad,Adquisicion_FK,Date_Key,Cod_INE,Nombre_y,Especificaciones,UnidadMedida,Tipo,Dimensional,Tipo_Adquisicion
0,1,1,1,7700,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,1,44966,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,Litro,Volumen,L,Contrato Abierto
1,2,1,2,3850,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,1,44966,264-002-000,PLAGUICIDA BIOLÓGICO BACILLUS SPHAERICUS,PLAGUICIDA BIOLÓGICO PARA CONTROL DE LARVAS AN...,Kilogramo,Peso,kg,Contrato Abierto
2,5,1,5,4554,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,1,44966,264-005-000,PLAGUICIDA BACILLUS THURINGIENSIS VAR. ISRAELE...,PLAGUICIDA BIOLÓGICO PARA EL CONTROL DE LARVAS...,Kilogramo,Peso,kg,Contrato Abierto


In [186]:
df_Catalogo_Evento_Producto_Medida_Ad = df_Catalogo_Evento_Producto_Medida_Ad.rename(columns={'Date_Key':'Date_key'})
df_Catalogo_Evento_Producto_Medida_Ad = df_Catalogo_Evento_Producto_Medida_Ad.drop('Date_key', axis=1)

In [188]:
df_Catalogo_Evento_Producto_Medida_Ad = df_Catalogo_Evento_Producto_Medida_Ad.drop('Adquisicion_FK', axis=1)

In [194]:
df_Catalogo_Evento_Producto_Medida_Ad.head(2)

,Catalogo_ID,Revision,Renglon,Cantidad,NOG,Nombre_x,Enlace,Entidad,Cod_INE,Nombre_y,Especificaciones,UnidadMedida,Tipo,Dimensional,Tipo_Adquisicion
0,1,1,1,7700,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,Litro,Volumen,L,Contrato Abierto
1,2,1,2,3850,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,264-002-000,PLAGUICIDA BIOLÓGICO BACILLUS SPHAERICUS,PLAGUICIDA BIOLÓGICO PARA CONTROL DE LARVAS AN...,Kilogramo,Peso,kg,Contrato Abierto


### Renombrando columna entidad

In [195]:
df_Catalogo_Evento_Producto_Medida_Ad = df_Catalogo_Evento_Producto_Medida_Ad.rename(columns={'Entidad':'Entidad_FK'})

In [196]:
df_Catalogo_Evento_Producto_Medida_Ad.head(1)

,Catalogo_ID,Revision,Renglon,Cantidad,NOG,Nombre_x,Enlace,Entidad_FK,Cod_INE,Nombre_y,Especificaciones,UnidadMedida,Tipo,Dimensional,Tipo_Adquisicion
0,1,1,1,7700,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,Litro,Volumen,L,Contrato Abierto


In [200]:
df_Catalogo_Evento_Producto_Medida_Ad_Ent = df_Catalogo_Evento_Producto_Medida_Ad.merge(df_Entidad, on='Entidad_FK', how='inner')
df_Catalogo_Evento_Producto_Medida_Ad_Ent.head(2)

,Catalogo_ID,Revision,Renglon,Cantidad,NOG,Nombre_x,Enlace,Entidad_FK,Cod_INE,Nombre_y,Especificaciones,UnidadMedida,Tipo_x,Dimensional,Tipo_Adquisicion,Nombre,Tipo_y,Sub_Tipo,Unidad_Compradora,timestamp
0,1,1,1,7700,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,Litro,Volumen,L,Contrato Abierto,MINISTERIO DE FINANZAS PÚBLICAS,Administración Central,None,None,2023-04-06 01:09:14
1,2,1,2,3850,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,2,264-002-000,PLAGUICIDA BIOLÓGICO BACILLUS SPHAERICUS,PLAGUICIDA BIOLÓGICO PARA CONTROL DE LARVAS AN...,Kilogramo,Peso,kg,Contrato Abierto,MINISTERIO DE FINANZAS PÚBLICAS,Administración Central,None,None,2023-04-06 01:09:14


### Declarando catàlogo final

In [208]:
Catalogo_final = df_Catalogo_Evento_Producto_Medida_Ad_Ent 
Catalogo_final.head(5)

,Catalogo_ID,Revision,Renglon,Cantidad,NOG,Nombre_x,Enlace,Cod_INE,Nombre_y,Especificaciones,UnidadMedida,Tipo_x,Dimensional,Tipo_Adquisicion,Nombre,Tipo_y,Sub_Tipo,Unidad_Compradora
0,1,1,1,7700,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,264-001-000,PLAGUICIDA DELTAMETRINA 4.5 % ULV,PLAGUICIDA PARA CONTROL DE INSECTOS VOLADORES ...,Litro,Volumen,L,Contrato Abierto,MINISTERIO DE FINANZAS PÚBLICAS,Administración Central,None,None
1,2,1,2,3850,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,264-002-000,PLAGUICIDA BIOLÓGICO BACILLUS SPHAERICUS,PLAGUICIDA BIOLÓGICO PARA CONTROL DE LARVAS AN...,Kilogramo,Peso,kg,Contrato Abierto,MINISTERIO DE FINANZAS PÚBLICAS,Administración Central,None,None
2,5,1,5,4554,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,264-005-000,PLAGUICIDA BACILLUS THURINGIENSIS VAR. ISRAELE...,PLAGUICIDA BIOLÓGICO PARA EL CONTROL DE LARVAS...,Kilogramo,Peso,kg,Contrato Abierto,MINISTERIO DE FINANZAS PÚBLICAS,Administración Central,None,None
3,3,1,3,3300,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,264-003-000,PABELLÓN IMPREGNADO CON ALFACIPERMETRINA,PABELLÓN IMPREGNADO (MOSQUITEROS) MALLA DE POL...,UNIDAD,Unidades,u,Contrato Abierto,MINISTERIO DE FINANZAS PÚBLICAS,Administración Central,None,None
4,4,1,4,367126,18558690,CONTRATO ABIERTO NÚMERO 04-2022 ADQUISICIÓN DE...,Perla,264-004-000,LARVICIDA TEMEFOS (TEMEPHOS),PLAGUICIDA PARA EL CONTROL DE LARVAS DE MOSQUI...,UNIDAD,Unidades,u,Contrato Abierto,MINISTERIO DE FINANZAS PÚBLICAS,Administración Central,None,None


### Construyendo Catalogo de precios